# Setting Pyspark in Colab

## Spark 설치 및 초기 설정
 Spark은 대용량 데이터 처리를 위한 분산 컴퓨팅 프레임워크입니다


In [2]:
!sudo apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.4/spark-3.2.4-bin-hadoop3.2.tgz
!tar xf spark-3.2.4-bin-hadoop3.2.tgz
!pip install -q findspark

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [4]:
# findspark를 사용해 Spark 초기화
import findspark
findspark.init("./spark-3.2.4-bin-hadoop3.2")

In [5]:
# SparkSession 생성
# SparkSession은 Spark 프로그램의 시작점이며, 모든 Spark 기능의 진입점입니다

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

# Word Count (gutenberg)

1. gutenberg.txt 파일 로드
2. 문장부호를 제거하고 소문자로 변환
3. 단어 단위로 분리
4. (단어, 1) 형태의 tuple 생성
5. 단어를 기준으로 결합하여 개수 세기
6. 상위 10개 결과 확인

In [7]:
# RDD를 사용한 단어 수 세기
# RDD(Resilient Distributed Dataset)는 Spark의 기본 데이터 구조입니다
# 여러 단계의 변환(transformation)을 통해 데이터를 처리합니다
wordcount = sc.textFile("gutenberg.txt") \
              .map(lambda x: x.replace(",", "").replace(".", "").replace("'", "").replace('"', "").lower()) \
              .flatMap(lambda x: x.split()) \
              .map(lambda x: (x, 1)) \
              .reduceByKey(lambda x, y: x + y) \
              .map(lambda x: (x[1], x[0])) \
              .sortByKey(False)

print(wordcount.take(10))

[(2848, 'the'), (1475, 'to'), (1457, 'of'), (1454, 'and'), (1093, 'i'), (982, 'a'), (815, 'in'), (771, 'that'), (608, 'he'), (579, 'was')]


## 한 줄씩 처리 과정 이해

In [8]:
# 1. 파일 읽기
# textFile()은 텍스트 파일을 읽어 RDD로 변환합니다
rdd1 = sc.textFile("gutenberg.txt")  # 파일로부터 rdd 만들기
rdd1.take(10)

['The Project Gutenberg EBook of Friday, the Thirteenth, by Thomas W. Lawson',
 '',
 'This eBook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  You may copy it, give it away or',
 're-use it under the terms of the Project Gutenberg License included',
 'with this eBook or online at www.gutenberg.net',
 '',
 '',
 'Title: Friday, the Thirteenth',
 '']

In [9]:
# 2. 문장부호 제거 및 소문자 변환
# map()은 각 요소에 함수를 적용하는 변환 연산입니다
rdd2 = rdd1.map(lambda x: x.replace(",", "").replace(".", "").replace("'", "").replace('"', "").lower())  # 문장부호 제거 및 소문자로
rdd2.take(10)

['the project gutenberg ebook of friday the thirteenth by thomas w lawson',
 '',
 'this ebook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever  you may copy it give it away or',
 're-use it under the terms of the project gutenberg license included',
 'with this ebook or online at wwwgutenbergnet',
 '',
 '',
 'title: friday the thirteenth',
 '']

In [10]:
# 3. 단어 단위로 분리
# flatMap()은 각 요소를 여러 개의 새로운 요소로 분리합니다
rdd3 = rdd2.flatMap(lambda x: x.split())  # 단어 단위 분리 (map이 아닌 flatMap을 사용한다는 점에 주목)
rdd3.take(10)

['the',
 'project',
 'gutenberg',
 'ebook',
 'of',
 'friday',
 'the',
 'thirteenth',
 'by',
 'thomas']

In [11]:
# 4. (단어, 1) 형태의 tuple 생성
# 각 단어를 키로 하고 값을 1로 하는 tuple을 만듭니다
rdd4 = rdd3.map(lambda x: (x, 1))  # reduceByKey를 사용하기 위해 (key, value)의 튜플 형태로 만듦. key=단어, value=1
rdd4.take(10)

[('the', 1),
 ('project', 1),
 ('gutenberg', 1),
 ('ebook', 1),
 ('of', 1),
 ('friday', 1),
 ('the', 1),
 ('thirteenth', 1),
 ('by', 1),
 ('thomas', 1)]

In [12]:
# 5. 단어별로 개수 합치기
# reduceByKey()는 같은 키를 가진 값들을 지정된 함수로 결합합니다
rdd5 = rdd4.reduceByKey(lambda x, y: x + y)  # reduceByKey를 사용하여 같은 key(단어)의 value(1)들을 모두 더함. 즉, 단어들의 등장 횟수를 구함
rdd5.take(10)

[('project', 89),
 ('gutenberg', 27),
 ('ebook', 13),
 ('of', 1457),
 ('friday', 34),
 ('thirteenth', 12),
 ('thomas', 10),
 ('lawson', 17),
 ('this', 236),
 ('is', 349)]

In [13]:
# 6. (개수, 단어) 형태로 변환
# 정렬을 위해 키와 값의 위치를 바꿉니다
rdd6 = rdd5.map(lambda x: (x[1], x[0]))  # sortByKey를 사용하기 위해 (key, value)의 튜플 형태로 만듦. key=단어 등장 빈도, value=단어
rdd6.take(10)

[(89, 'project'),
 (27, 'gutenberg'),
 (13, 'ebook'),
 (1457, 'of'),
 (34, 'friday'),
 (12, 'thirteenth'),
 (10, 'thomas'),
 (17, 'lawson'),
 (236, 'this'),
 (349, 'is')]

In [14]:
# 7. 개수를 기준으로 내림차순 정렬
# sortByKey()는 키를 기준으로 정렬합니다
rdd7 = rdd6.sortByKey(False)  # sortByKey를 사용하여 key 크기 순으로 정렬. 인자로 False를 주면 내림차순 정렬, 인자로 True를 주면 오름차순 정렬.
rdd7.take(10)  # 최빈 단어 10개 출력

[(2848, 'the'),
 (1475, 'to'),
 (1457, 'of'),
 (1454, 'and'),
 (1093, 'i'),
 (982, 'a'),
 (815, 'in'),
 (771, 'that'),
 (608, 'he'),
 (579, 'was')]